### Author: Ryan Inghilterra - DSE 210

### Homework 4 - Worksheet 8 - Generative Models 3 - Problem 3 - Text Classification using Multinomial Bayes


(a) For this problem, you’ll be using the 20 Newsgroups data set. There are several versions of it on
the web. You should download “20news-bydate.tar.gz” from
http://qwone.com/~jason/20Newsgroups/
Unpack it and look through the directories at some of the files. Overall, there are roughly 19,000
documents, each from one of 20 newsgroups. The label of a document is the identity of its
newsgroup. The documents are divided into a training set and a test set.    

(b) The same website has a processed version of the data, “20news-bydate-matlab.tgz”, that is particularly
convenient to use. Download this and also the file “vocabulary.txt”. Look at the first
training document in the processed set and the corresponding original text document to understand
the relation between the two.

(c) The words in the documents constitute an overall vocabulary V of size 61188. Build a multinomial
Naive Bayes model using the training data. For each of the 20 classes j = 1, 2,..., 20, you must
have the following:
• ⇡j , the fraction of documents that belong to that class; and
• Pj , a probability distribution over V that models the documents of that class.
In order to fit Pj , imagine that all the documents of class j are strung together. For each word
w 2 V , let Pjw be the fraction of this concatenated document occupied by w. Well, almost: you
will need to do smoothing (just add one to the count of how often w occurs

In [1]:
import pandas as pd
import numpy as np

read in train data

In [2]:
train_data = pd.read_table('20news-bydate_matlab/matlab/train.data', delim_whitespace=True, header=None)
train_label = pd.read_table('20news-bydate_matlab/matlab/train.label', delim_whitespace=True, header=None).reset_index()
print(train_data.shape)
print(train_label.shape)
train_data.columns = [ 'doc', 'word', 'word_freq']
train_label.columns = [ 'doc', 'label']
train_label['doc'] = train_label['doc'] + 1 #fix indexing to match doc number
train_label.head()
# merge train_data and train_label into train_set
train_set = pd.merge(train_data, train_label, how='outer', on='doc')
print(train_set.shape)
vocabulary = pd.read_table('vocabulary.txt', header=None)
print(vocabulary.shape)
vocab_len = len(vocabulary)
print(vocab_len)

(1467345, 3)
(11269, 2)
(1467345, 4)
(61188, 1)
61188


In [3]:
train_set.tail()

,doc,word,word_freq,label
1467340,11269,47387,1,20
1467341,11269,48339,1,20
1467342,11269,48919,1,20
1467343,11269,51544,1,20
1467344,11269,53958,1,20


calculate the log of class probs (log(pi_j)) (the fraction of documents that belong to each class)

In [4]:
pi_probs = train_set.groupby('label').count().iloc[:,0] / len(train_set)  # divide by total number of observations
log_pi_probs = np.log(pi_probs)
log_pi_probs_dic = log_pi_probs.to_dict()
log_pi_probs_dic

{1: -3.0010518984537291,
 2: -3.2226619171399928,
 3: -3.2871001103876947,
 4: -3.236438137215806,
 5: -3.31876937733667,
 6: -3.066246540224062,
 7: -3.5469903751709828,
 8: -3.0526784417834851,
 9: -3.093677204299814,
 10: -3.1094499000930336,
 11: -2.9462104104812785,
 12: -2.6888135818793875,
 13: -3.1730186356217027,
 14: -2.8437935683802684,
 15: -2.8565315433703695,
 16: -2.6799880865682248,
 17: -2.7630395802188601,
 18: -2.5609236553068171,
 19: -2.8575400564923634,
 20: -3.2040410458450785}

• Pj , a probability distribution over V that models the documents of that class.
In order to fit Pj , imagine that all the documents of class j are strung together. For each word
w 2 V , let Pjw be the fraction of this concatenated document occupied by w. Well, almost: you
will need to do smoothing (just add one to the count of how often w occurs

In [5]:
train_set.head()

,doc,word,word_freq,label
0,1,1,4,1
1,1,2,2,1
2,1,3,10,1
3,1,4,4,1
4,1,5,2,1


we want to combine all documents to have word counts per class instead (treat each class as one giant doc)

In [6]:
combined_class_df_list = []
empty_df = pd.DataFrame(1, index=np.arange(1,vocab_len+1), columns=['word_freq']).reset_index()
empty_df.columns = ['word', 'word_freq']

group_df = train_set.groupby(['label','word']).sum().reset_index()
group_df = group_df[['label', 'word', 'word_freq']]
group_df = group_df.groupby(['label', 'word']).sum().reset_index()
for i in range(1,21):
    a_df = group_df[group_df['label'] == i].reset_index()[['word', 'word_freq']]
    merged = pd.merge(empty_df, a_df, how='left', on='word')
    merged = merged[['word','word_freq_y']]
    merged.columns = ['word', 'word_freq']
    merged['word_freq'] = merged['word_freq'].fillna(value=0)
    merged = merged.set_index('word')
    combined_class_df_list.append(merged['word_freq'])
print(len(combined_class_df_list))

20


In [7]:
class_full_df = pd.DataFrame(combined_class_df_list)
class_full_df = class_full_df.reset_index(drop=True)
class_full_df

word,1,2,3,4,5,6,7,8,9,10,...,61179,61180,61181,61182,61183,61184,61185,61186,61187,61188
0,13.0,63.0,275.0,9.0,82.0,41.0,6.0,1.0,34.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,60.0,59.0,0.0,17.0,14.0,58.0,10.0,6.0,172.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11.0,69.0,0.0,17.0,21.0,34.0,2.0,2.0,144.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.0,31.0,0.0,0.0,10.0,48.0,2.0,1.0,48.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.0,33.0,0.0,1.0,1.0,47.0,1.0,0.0,47.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,47.0,222.0,0.0,79.0,15.0,52.0,21.0,3.0,237.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,28.0,0.0,2.0,2.0,32.0,0.0,0.0,28.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,9.0,54.0,0.0,0.0,13.0,86.0,7.0,3.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,14.0,67.0,0.0,4.0,4.0,83.0,3.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,33.0,0.0,2.0,1.0,302.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


the class_full_df has the 20 rows, for each class, and a column for each word (61188) total, where the value represents the word frequency

In [8]:
# need to divide by (total # words per class) + vocab_len - this is the denominator to divide the pjx df by
total_word_counts = class_full_df.sum(axis=1) + vocab_len
total_word_counts

0     210000.0
1     171546.0
2     151955.0
3     160334.0
4     147378.0
5     214034.0
6     122282.0
7     175290.0
8     163819.0
9     169086.0
10    202455.0
11    261644.0
12    164361.0
13    216526.0
14    214902.0
15    262455.0
16    237102.0
17    315993.0
18    247614.0
19    180284.0
dtype: float64

In [9]:
pd.set_option('precision', 10) # do not want to lose precision
pjx_df = class_full_df + 1 # smoothing add 1
pjx_df = pjx_df.divide(total_word_counts, axis=0) #divide each row by the total_word_counts (denominator)
log_pjx_df = pjx_df.apply(np.vectorize(np.log)) #np.log applied to each pjx (which is each row)
log_pjx_df.head()

word,1,2,3,4,5,6,7,8,9,10,...,61179,61180,61181,61182,61183,61184,61185,61186,61187,61188
0,-9.6158054801,-8.0959797263,-6.6344619440,-9.9522777167,-7.8360222019,-8.5171931914,-10.3089526606,-11.5617156291,-8.6995147482,-7.3061029193,...,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097,-12.2548628097
1,-7.9417328670,-7.9582621690,-12.0526067312,-9.1622349733,-9.3445565301,-7.9750692873,-9.6547114584,-10.1066965821,-6.8993151367,-10.6663123701,...,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312,-12.0526067312
2,-9.4464330536,-7.6828444613,-11.9313397034,-9.0409679455,-8.8402972500,-8.3759916419,-10.8327274147,-10.8327274147,-6.9546059609,-11.9313397034,...,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034,-11.9313397034
3,-9.7877898411,-8.5192785156,-11.9850144184,-11.9850144184,-9.5871191456,-8.0931941203,-10.8864021297,-11.2918672379,-8.0931941203,-11.9850144184,...,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184,-11.9850144184
4,-9.9548458448,-8.3743954693,-11.9007559939,-11.2076088133,-11.2076088133,-8.0295549830,-11.2076088133,-11.9007559939,-8.0295549830,-11.9007559939,...,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939,-11.9007559939


In [10]:
log_pjx_transpose = log_pjx_df.transpose()
log_pjx_transpose.columns = range(1,21)
log_pjx_transpose.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
word,,,,,,,,,,,,,,,,,,,,
1,-9.6158054801,-7.9417328670,-9.4464330536,-9.7877898411,-9.9548458448,-8.4026891490,-11.7140851318,-9.7716119312,-9.2984672377,-11.3450155597,-11.5251257381,-8.5044481666,-10.6235261461,-9.5128771894,-8.3861170911,-12.4778349177,-9.3805134337,-10.2655800676,-12.4196263610,-12.1022886643
2,-8.0959797263,-7.9582621690,-7.6828444613,-8.5192785156,-8.3743954693,-6.8667183884,-8.3467893018,-8.0668638390,-7.7870097336,-8.5118022156,-7.9987652134,-7.9638805736,-8.4834599826,-8.5965864575,-7.8590967814,-7.6575533521,-8.8498851827,-7.6200502235,-8.7307469069,-8.2736472678
3,-6.6344619440,-12.0526067312,-11.9313397034,-11.9850144184,-11.9007559939,-12.2738901599,-11.7140851318,-12.0741970242,-12.0065174388,-12.0381627402,-12.2182729186,-12.4747400801,-12.0098205072,-12.2854659116,-12.2779373892,-9.6446215737,-12.3762457073,-12.6634753404,-12.4196263610,-9.7997035713
4,-9.9522777167,-9.1622349733,-9.0409679455,-11.9850144184,-11.2076088133,-7.8918635252,-10.6154728431,-12.0741970242,-10.3970795264,-10.9395504515,-12.2182729186,-9.9898334304,-10.9112082185,-9.6464085820,-9.1424431733,-10.3983933761,-10.4303355582,-10.3608902474,-9.2415725307,-11.0036763756
5,-7.8360222019,-9.3445565301,-8.8402972500,-9.5871191456,-11.2076088133,-9.5013014376,-10.6154728431,-9.4351396946,-10.3970795264,-11.3450155597,-11.5251257381,-8.3803955179,-10.2180610380,-9.2409434739,-9.7129880318,-9.7697847166,-9.8913390575,-11.5648630518,-9.5292546031,-8.9242348339


at this point we have log(Pjx) transpose above (or log(Pjw) since we are talking in terms of words)

lets read in our test data which we will use later for part (e)

In [11]:
test_data = pd.read_table('20news-bydate_matlab/matlab/test.data', delim_whitespace=True, header=None)
test_label = pd.read_table('20news-bydate_matlab/matlab/test.label', delim_whitespace=True, header=None).reset_index()
print(test_data.shape)
print(test_label.shape)
test_data.columns = [ 'doc', 'word', 'word_freq']
test_label.columns = [ 'doc', 'label']
test_label['doc'] = test_label['doc'] + 1
test_label.head()
# merge test_data and test_label into test_set
test_set = pd.merge(test_data, test_label, how='outer', on='doc')
print(test_set.shape)

(967874, 3)
(7505, 2)
(967874, 4)


In [12]:
test_set.head()

,doc,word,word_freq,label
0,1,3,1,1
1,1,10,1,1
2,1,12,8,1
3,1,17,1,1
4,1,23,8,1


(d) routine that uses naive baiyes model to classify document (routine below done on test set)

In [13]:
# doc_df which has words and word_freqs and the log(class_probs) and log_pjx_df(each row a word, each column a class)
def naive_bayes(doc_df, log_pi_probs, log_pjx_df):
    # merge the test set with the pjx probs
    sub_df = pd.merge(doc_df[['doc','word_freq']], log_pjx_df, left_index=True, right_index=True, how='left')
    #multiply the pjx probs value * the word frequencies
    sub_df_multiplied = sub_df[range(1,21)].multiply(sub_df['word_freq'], axis="index")
    pjx_vals = sub_df_multiplied.sum(axis=0) # sum up the pjx probs for each word to sum to doc
    bayes_args = pjx_vals + log_pi_probs #add the log_pjx values + log(class probability)
    predict_label = bayes_args.idxmax() # select the label which gave the largest of these values
    return predict_label

(e) Evaluate the performance of your model on the test data. What error rate do you achieve?

In [15]:
%%time
log_pi_probs = pd.Series(log_pi_probs_dic)
doc_predict_dic = {}
num_of_docs = len(test_set['doc'].unique())
# loop through each document
for i in range(1,num_of_docs + 1):
    a_doc = test_set[test_set['doc'] == i].set_index('word')
    predict_label = naive_bayes(a_doc, log_pi_probs, log_pjx_transpose) #need to pass in the transpose (words as rows)
    doc_predict_dic[i] = predict_label

CPU times: user 28.4 s, sys: 172 ms, total: 28.6 s
Wall time: 28.5 s


the test set is fully classified by my naive_bayes routine in about 30 seconds

In [16]:
final_df = pd.DataFrame(doc_predict_dic, index=[0]).transpose()
final_df.columns = ['predict_label']
print(final_df.shape)
final_df.head()

(7505, 1)


,predict_label
1,1
2,1
3,1
4,16
5,1


In [17]:
final_results = test_label.set_index('doc').merge(final_df, left_index=True, right_index=True)
print(final_results.shape)
final_results.head()

(7505, 2)


,label,predict_label
doc,,
1,1,1
2,1,1
3,1,1
4,1,16
5,1,1


In [18]:
num_predicted_correct = len(final_results[final_results['predict_label'] == final_results['label']])
print(num_predicted_correct)

5848


In [19]:
test_set_error = 1 - (num_predicted_correct / float(len(final_results)))
test_set_error

0.22078614257161888

**~22% error rate on test set (in other words 78% of the test documents were classified correctly)**